In [4]:
from keras.models import Sequential
from keras.layers import *
import numpy as np
from sklearn.model_selection import train_test_split
import glob

import sys
sys.path.append('/anaconda3/lib/python3.5/site-packages')

import cv2

Using TensorFlow backend.


In [6]:
img_width = 600
img_height = 800
alpha = 0.001

model = Sequential()
model.add(Conv2D(16, (3, 3), padding='same', strides=(2, 2), input_shape=(img_width, img_height, 3)))
model.add(LeakyReLU(alpha=alpha))
model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Dropout(0.2))

model.add(Conv2D(32, (3, 3), padding='same'))
model.add(LeakyReLU(alpha=alpha))
model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Dropout(0.2))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(LeakyReLU(alpha=alpha))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(128, activation='tanh'))
model.add(Dropout(0.3))
model.add(Dense(3, activation='softmax'))


In [7]:
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 300, 400, 16)      448       
_________________________________________________________________
leaky_re_lu_4 (LeakyReLU)    (None, 300, 400, 16)      0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 100, 133, 16)      0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 100, 133, 16)      0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 100, 133, 32)      4640      
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 100, 133, 32)      0         
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 33, 44, 32)        0         
__________

In [8]:
# Image augmentation from https://github.com/vxy10/ImageAugmentation/blob/master/img_transform_NB.ipynb

def augment_brightness_camera_images(image):
    image1 = cv2.cvtColor(image,cv2.COLOR_RGB2HSV)
    random_bright = .25+np.random.uniform()
    #print(random_bright)
    image1[:,:,2] = image1[:,:,2]*random_bright
    image1 = cv2.cvtColor(image1,cv2.COLOR_HSV2RGB)
    return image1

def transform_image(img,ang_range,shear_range,trans_range):
    '''
    This function transforms images to generate new images.
    The function takes in following arguments,
    1- Image
    2- ang_range: Range of angles for rotation
    3- shear_range: Range of values to apply affine transform to
    4- trans_range: Range of values to apply translations over. 
    
    A Random uniform distribution is used to generate different parameters for transformation
    
    '''
    # Rotation

    ang_rot = np.random.uniform(ang_range)-ang_range/2
    rows,cols,ch = img.shape    
    Rot_M = cv2.getRotationMatrix2D((cols/2,rows/2),ang_rot,1)

    # Translation
    tr_x = trans_range*np.random.uniform()-trans_range/2
    tr_y = trans_range*np.random.uniform()-trans_range/2
    Trans_M = np.float32([[1,0,tr_x],[0,1,tr_y]])

    # Shear
    pts1 = np.float32([[5,5],[20,5],[5,20]])

    pt1 = 5+shear_range*np.random.uniform()-shear_range/2
    pt2 = 20+shear_range*np.random.uniform()-shear_range/2
    
    # Brightness 
    
    pts2 = np.float32([[pt1,5],[pt2,pt1],[5,pt2]])

    shear_M = cv2.getAffineTransform(pts1,pts2)
        
    img = cv2.warpAffine(img,Rot_M,(cols,rows))
    img = cv2.warpAffine(img,Trans_M,(cols,rows))
    img = cv2.warpAffine(img,shear_M,(cols,rows))
    
    img = augment_brightness_camera_images(img)
    
    return img

In [9]:
X_data = []
y_data = []
red = [1,0,0]
green = [0,1,0]
yellow = [0,0,1]

files = glob.glob("/Users/willbuxton/Downloads/Augmentated Dataset/TrafficLight_additional/red/*.png")
print(len(files))
i = 0
for file in files:
    i += 1
    img = cv2.imread(file)
    #img = cv2.resize(img, (img_width, img_height))
    X_data.append(img)
    y_data.append(red)
    image = transform_image(img,7,4,2)
    X_data.append(image)
    y_data.append(red)

files = glob.glob("/Users/willbuxton/Downloads/Augmentated Dataset/TrafficLight_additional/ylw/*.png")
print(len(files))
for file in files: 
    img = cv2.imread(file)
    #img = cv2.resize(img, (img_width, img_height))
    X_data.append(img)
    y_data.append(yellow)
    for i in range(20):
        image = transform_image(img,7,4,2)
        X_data.append(image)
        y_data.append(yellow)

files = glob.glob("/Users/willbuxton/Downloads/Augmentated Dataset/TrafficLight_additional/grn/*.png")
print(len(files))
for file in files:
    img = cv2.imread(file)
    #img = cv2.resize(img, (img_width, img_height))
    X_data.append(img)
    y_data.append(green)
    for i in range(10):
        image = transform_image(img,7,4,2)
        X_data.append(image)
        y_data.append(yellow)

X = np.array(X_data)
y = np.array(y_data)

assert(X.shape[0] == y.shape[0])
print(X.shape)

6072


KeyboardInterrupt: 

In [25]:
np.save("X.npy", X)
np.save("y.npy", y)

In [4]:
X_train = np.load("X.npy")
y_train = np.load("y.npy")

In [5]:
print(X_train.shape)
print(y_train.shape)

(5035, 600, 800, 3)
(5035, 3)


In [ ]:
model.compile(optimizer='rmsprop', loss="categorical_crossentropy", metrics=['accuracy'])

X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
model.fit(X_train, y_train, validation_split=0.2, shuffle=True, epochs=10, batch_size=4, verbose=1)


Train on 3222 samples, validate on 806 samples
Epoch 1/10
 324/3222 [==>...........................] - ETA: 24:30:32 - loss: 15.0409 - acc: 0.0556

In [ ]:
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [13]:
model.save("model.h5")